In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 證交所api

In [49]:
# 股票列表
#台指：^TWII、電子：^TELI
#群聯 8299 台積電 2330 瑞昱 2379
stock_symbols = ["8299","2330","2379","^TWII"]


In [93]:
import requests
import json

stockNo = "2330"

tw_api = "https://www.twse.com.tw/rwd/zh/afterTrading/STOCK_DAY?202409&stockNo=2330&response=json"

data = requests.get(tw_api)

stock_info = json.loads(data.text)
stock_info

{'stat': 'OK',
 'date': '20240919',
 'title': '113年09月 2330 台積電           各日成交資訊',
 'fields': ['日期', '成交股數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '成交筆數'],
 'data': [['113/09/02',
   '19,272,593',
   '18,270,058,260',
   '950.00',
   '955.00',
   '943.00',
   '948.00',
   '+4.00',
   '31,642'],
  ['113/09/03',
   '23,205,623',
   '21,908,471,541',
   '948.00',
   '952.00',
   '939.00',
   '940.00',
   '-8.00',
   '47,139'],
  ['113/09/04',
   '93,169,835',
   '83,424,133,824',
   '894.00',
   '905.00',
   '888.00',
   '889.00',
   '-51.00',
   '393,210'],
  ['113/09/05',
   '34,147,890',
   '30,998,595,394',
   '907.00',
   '915.00',
   '900.00',
   '902.00',
   '+13.00',
   '63,902'],
  ['113/09/06',
   '28,248,063',
   '25,786,016,936',
   '909.00',
   '918.00',
   '903.00',
   '918.00',
   '+16.00',
   '38,415'],
  ['113/09/09',
   '38,448,946',
   '34,456,838,126',
   '892.00',
   '900.00',
   '891.00',
   '899.00',
   '-19.00',
   '102,365'],
  ['113/09/10',
   '34,312,646',

In [90]:
#對json作處理
tsmc = pd.DataFrame(stock_info['data'], columns=stock_info['fields'])

In [94]:
tsmc

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
0,113/09/02,"19,272,593","18,270,058,260",950.00,955.00,943.00,948.00,+4.00,"31,642"
1,113/09/03,"23,205,623","21,908,471,541",948.00,952.00,939.00,940.00,-8.00,"47,139"
2,113/09/04,"93,169,835","83,424,133,824",894.00,905.00,888.00,889.00,-51.00,"393,210"
3,113/09/05,"34,147,890","30,998,595,394",907.00,915.00,900.00,902.00,+13.00,"63,902"
4,113/09/06,"28,248,063","25,786,016,936",909.00,918.00,903.00,918.00,+16.00,"38,415"
5,113/09/09,"38,448,946","34,456,838,126",892.00,900.00,891.00,899.00,-19.00,"102,365"
6,113/09/10,"34,312,646","31,067,424,234",907.00,911.00,901.00,904.00,+5.00,"45,141"
7,113/09/11,"19,513,256","17,621,000,956",906.00,906.00,900.00,901.00,-3.00,"30,789"
8,113/09/12,"43,749,260","40,999,951,508",936.00,944.00,928.00,940.00,X0.00,"80,643"
9,113/09/13,"28,307,441","26,795,862,243",955.00,955.00,939.00,947.00,+7.00,"39,071"


# 從yahoo爬蟲

In [95]:
from bs4 import BeautifulSoup

In [106]:
# 使用 BeautifulSoup 讀取
yahoo_url="https://tw.stock.yahoo.com/quote/2330.TW"
context = requests.get(yahoo_url)
soup = BeautifulSoup(context.text,'html.parser')

# 尋找指定 class 的 <ul> 標籤
ul_tag = soup.find('ul', class_='D(f) Fld(c) Flw(w) H(192px) Mx(-16px)')

In [143]:
span_items = ul_tag.find_all('span')
y_fields = []
y_data =[]
tmp = 0
for item in span_items:
    
    if item.text == '':
        tmp+=2
        continue
    print(item.text)
    if tmp%2==0:
        y_fields.append(item.text)
    else:
        y_data.append(item.text)
    tmp+=1

成交
960
開盤
940
最高
960
最低
936
均價
949
成交金額(億)
310.27
昨收
941
漲跌幅
2.02%
漲跌
19.00
總量
32,674
昨量
28,394
振幅
2.55%


['960',
 '940',
 '960',
 '936',
 '949',
 '310.27',
 '941',
 '2.02%',
 '19.00',
 '32,674',
 '28,394',
 '2.55%']

In [144]:
yfinance = pd.DataFrame([y_data],columns=y_fields)

In [145]:
yfinance

,成交,開盤,最高,最低,均價,成交金額(億),昨收,漲跌幅,漲跌,總量,昨量,振幅
0,960,940,960,936,949,310.27,941,2.02%,19.00,"32,674","28,394",2.55%
